# 🧠 BCI Competition: SSVEP Inference Template

Welcome, teams\! This notebook is your template for the real-time inference competition.

### Your Task

You are required to modify **two specific functions**: `load_model` and `predict`. The surrounding infrastructure for data recording and game communication is fixed and must not be altered.

1.  Add any necessary library imports in **Part 1**.
2.  Complete the `load_model` function in **Part 3**.
3.  Complete the `predict` function in **Part 4**. This is where you will implement your custom preprocessing and prediction logic.
4.  Submit this completed notebook and your model file(s).


## Part 1: Team-Specific Imports <span style="color:green">*(✍️ EDITABLE)*</span>.

In the cell below, add any libraries required to load your model and process the data (e.g., `tensorflow`, `sklearn`, `mne`, `scipy`).


In [3]:
# ==> YOUR CODE HERE <==
# Example:
# import mne
# from scipy.signal import butter, lfilter
# import joblib
from model.MTCformerV3 import MTCFormer
import torch
from utils.preprocessing import SignalPreprocessor
import numpy as np 
from utils.training import predict_optimized
print("Team-specific libraries would be imported here.")


Team-specific libraries would be imported here.


## Part 2: Data Recording Infrastructure <span style="color:red">*(❌ DO NOT EDIT or ADD anything)*</span>.

The system uses a dedicated function to record each trial of raw EEG data from the headset. This function captures the data and passes it to your `predict()` function as a pandas DataFrame. You do not need to modify this.

```python
def record_trial(inlet):
    # This function's internal logic is fixed.
    # It records 7 seconds of data and returns a DataFrame.
    ...
```

The structure of the pandas DataFrame given to `predict()` is the same as kaggle dataset but for only one **TRIAL** with **1750 row** which is the total number of samples for single trial in SSVEP.

```c++
              Time            FZ            C3           CZ  ...     Gyro3    Battery  Counter  Validation
0     1.664401e+06  332287.40625  357817.12500  640024.6250  ...  0.366211  66.666672  19733.0         1.0
1     1.664401e+06  335842.93750  361377.65625  647339.8750  ...  0.274658  66.666672  19734.0         1.0
2     1.664401e+06  337096.59375  361047.75000  650981.2500  ...  0.030518  66.666672  19735.0         1.0
3     1.664401e+06  334265.53125  357220.50000  645839.3125  ...  0.122070  66.666672  19736.0         1.0
4     1.664401e+06  331264.84375  355187.93750  638970.1250  ...  0.091553  66.666672  19737.0         1.0
...            ...           ...           ...          ...  ...       ...        ...      ...         ...
1745  1.664410e+06  331389.93750  356450.34375  629658.8750  ...  0.701904  66.666672  21978.0         1.0
1746  1.664410e+06  334618.59375  360340.09375  636000.7500  ...  0.671387  66.666672  21979.0         1.0
1747  1.664410e+06  336978.75000  361447.93750  641773.7500  ...  0.671387  66.666672  21980.0         1.0
1748  1.664410e+06  335227.37500  358243.93750  639073.8750  ...  0.732422  66.666672  21981.0         1.0
1749  1.664410e+06  331707.93750  355082.68750  631451.1250  ...  0.823975  66.666672  21982.0         1.0
```

## Part 3: Model Loading <span style="color:green">*(✍️ EDITABLE)*</span>.

Complete this function to load your trained model from the provided file path.

**Instructions:**

  * The function takes a `model_path` string as input.
  * It must load your model and return the model object.
  * Ensure the model is in evaluation/inference mode (e.g., `model.eval()`) which depends on your modeling.


In [4]:
def load_model(model_path):
    """
    Load the trained SSVEP model from the given file path.

    Args:
        model_path (str): The path to the model file.

    Returns:
        model: The loaded model object.
    """
    model = MTCFormer(
        depth=5,
        kernel_size=50,
        n_times=500,
        chs_num=7,
        eeg_ch_nums=4,
        class_num=4,
        class_num_domain=30,
        modulator_kernel_size=30,
        domain_dropout=0.7,
        modulator_dropout=0.7,
        mid_dropout=0.7,
        output_dropout=0.7,
        k=100,
        projection_dimention=2,
        seed = 5445
    )
    model.load_state_dict(
        torch.load(model_path, weights_only=False)['model_state_dict'] , strict=False
        )
    model.eval()
    return model

## Part 4: Preprocessing and Prediction <span style="color:green">*(✍️ EDITABLE)*</span>.

Complete this function to process the raw EEG **TRIAL** and make a final prediction.

**Instructions:**

  * The function receives the `model` object (from `load_model`) and a `df` (a pandas DataFrame).
  * The `df` contains the **raw trial data** directly from the EEG recorder of total **1750 samples** which was the output of **7 sec** of sampling rate **250 hz**. You must perform all necessary **preprocessing** on this DataFrame inside this function.
  * The function must return one of two specific strings: `"L"`, `"R"`, `"F"`and `"B"`.
  * You can decide whether to send the prediction to the microcontroller or not, so if you are **NOT CONFIDENT ENOUGH** about your decision then send `"?"`

In [ ]:
def predict(model, df):
    """
    Preprocess the raw MI data and make a prediction.

    Args:
        model: The loaded model object.
        df (pd.DataFrame): The raw trial data.

    Returns:
        str: The prediction, which must be "left", "right", or "?".
    """
    def preprocess_optimized(
            trial_df,
            signal_processer = None
            ):
        eeg_col =  ['OZ', 'PO7', 'PO8', 'PZ']


        input_array = trial_df.drop(columns = ["Time" , "Battery" , "Counter"])[eeg_col+['AccX',
            'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3' , 'Validation']].to_numpy().T
        
        acc_channel = np.linalg.norm(input_array[4:7,:],axis = 0)
        gyro_channel = np.linalg.norm(input_array[7:10,:],axis = 0)
        Validation_Channel = input_array[10,:]
        input_array[4,:] = acc_channel
        input_array[5,:] = gyro_channel
        input_array[6,:] = Validation_Channel

        input_array = input_array[:7,:]


        preprocessed_test_data , _ , _ , weights_test = signal_processer.apply_preprocessing(np.expand_dims(input_array,axis=0), np.array([None]) , np.array([1]))
        
        num_windows_per_trial = signal_processer.num_windows_per_trial

        
        return (
            torch.from_numpy(preprocessed_test_data).to(torch.float32),
           -1,
            -1,
            torch.from_numpy(weights_test).to(torch.float32),
            num_windows_per_trial
            )

    preprocessor = SignalPreprocessor(
    fs=250,                                                 
    bandpass_low=8,                     
    bandpass_high=14,                  
    n_cols_to_filter=4,                   
    window_size=500,                      
    window_stride=50,                    
    idx_to_ignore_normalization=-1,        
    crop_range=(1.5 , 6)            
)
    data , _ , _ , weights , windows_per_trial = preprocess_optimized(df,signal_processer=preprocessor)

    
    index_to_label = {
        0: "Backward",
        1: "Forward",
        2: "Left",
        3: "Right"
    }

    prediction = predict_optimized(
        model=model,
        windows_per_trial=windows_per_trial,
        loader= (data , weights)

    )

    return index_to_label[prediction[0]]

## Part 5: Hardware Interface <span style="color:red">*(❌ DO NOT EDIT or ADD anything)*</span>.

After your `predict` function returns a command (e.g., `"F"`), the system passes this string to a hardware interface function. This function translates your prediction into a signal that is sent to the wheelchair's microcontroller (Arduino).

```python
def send_to_microcontroller(serial, prediction):
    # This function's internal logic is fixed.
    # It converts your prediction string to a character ('F', 'L', etc.)
    # and sends it over the serial port to the Arduino.
    ...